PYTHON NOTEBOOK USED TO ANSWER TO EXERCISES OF CHAPTER 2 OF MATH80624 LECTURE NOTES

Modified by:
1. Chun Peng (Created for RSOME January 2021)
2. Erick Delage (January 2021)

As discussed in Chapter 2 of the  [lecture notes](http://web.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal. 

WARNING!!!

The following code exploits a free Mosek licence for the course MATH80624 at HEC Montréal (expiration June 1st 2021). If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi. Otherwise, we recommend that you obtain your own licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

**Jean-Sébastien Matte**

**Sena Onen Oz**

# **Preliminaries**

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/erickdelage/80624
!cp ./80624/mosek.lic .
!rm -r ./80624
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

     |████████████████████████████████| 10.1MB 1.1MB/s 
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into '80624'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface


### Load the data

For each of the robust counterparts models presented below, derive a tractable linear programming reformulation and implement in RSOME using Python both in its reduced and unreduced form. In your implementations, assume that $n=150$ and that
\begin{align*}
&c_i := 0.15+i\frac{0.05}{150} &&a_i := 0\,, && A_{ij}:= \left\{\begin{array}{cl} \frac{0.05}{450}\sqrt{2in(n+1)} & \mbox{if i=j}\\ 0 & \mbox{otherwise} \end{array}\right.&& b := 0.02\;.
\end{align*}
Also, for exercises 2.1 and 2.3 consider that each $\bar{z}_i\in\mathbb{R}^n$, $i=1,\dots,150$ is composed as follows:
$$ \bar{z}_i = \frac{0.05}{450}\sqrt{2in(n+1)} e_i\,,$$
where each $e_i$ refers to the $i$-th column of the identity matrix, 
and that $\Gamma=4$ in exercise 2.2 while $\alpha= 0.5$ in exercise 2.3.

Note that the $\bar{z}_i$'s are loaded in the matrix form:
$$ \bar{\textbf{Z}}:=[\bar{z}_1\;\bar{z}_2\;\dots\;\bar{z}_K].$$

In [ ]:
n = 150
m = n
c = 0.15+(0.05/150)*np.arange(1,n+1)
a = np.zeros(n) 
b = 0.02
maxDev = (0.05/450)*np.sqrt(2*n*(n+1)*np.arange(1,n+1))

#For Convex Hull uncertainty set
Zbars = np.diag(maxDev) #[zbar_1 zbar_2 zbar_3 ... zbar_K]
K = Zbars.shape[1]

#For budgeted uncertainty set
Gamma = 4

#For CVaR uncertainty set
CVaR_alpha = 0.5

# **Exercise 2.1)**

We are interested in the convex hull uncertainty set
\begin{align}
\max_{x}\;\; & c^T x\\
\text{subject to}\;\; & (a+z)^Tx \leq b \,,\,\forall z\in\mathcal{Z}\\
& 0 \leq x \leq 1\;,
\end{align}
where
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^n\,\middle|\, \exists \theta\in\mathbb{R}^K,\, z = \sum_{i=1}^K \theta_i \bar{z}_i,\, \theta\geq 0, \, \sum_{i=1}^K \theta_i = 1\right\}$$


In [ ]:
model1_raw = ro.Model('Raw_RobustCounterpart_ConvexHull')  

# Define decision variable(s)
x1_raw = model1_raw.dvar(n)

# Define uncertain variable
z1_raw = model1_raw.rvar(n)
t1_raw = model1_raw.rvar(K)

# Define uncertainty set
Z1_raw = (z1_raw == (Zbars @ t1_raw), t1_raw >= 0, sum(t1_raw) == 1) 

# Objective function
model1_raw.max(c @ x1_raw)

# Constraints
model1_raw.st(((a + z1_raw) @ x1_raw <= b).forall(Z1_raw))
model1_raw.st(x1_raw >= 0)
model1_raw.st(x1_raw <= 1)

# Solve mdoel
model1_raw.solve(my_solver)

print('The objective of Raw Robust Counterpart is {0:0.4f}'.format(model1_raw.get()))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0200s
The objective of Raw Robust Counterpart is 3.2742


We reformulate the above problem into a reduced tractable linear reformulation as follows:

First rewrite the uncertainty set as 
\begin{equation*}
  \mathcal{Z}_{1} := \left\{ z = \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \in \mathbb{R}^n \middle|\, \theta \geq 0, \sum_{i = 1}^{K} \theta_{i} = 1 \right\}
\end{equation*}

which we rewrite in the familiar form of $Wz \leq v$ giving
\begin{equation*}
  \mathcal{Z}_{1} := \left\{ \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \in \mathbb{R}^n \middle|\, \begin{bmatrix} -I \in \mathbb{R}^{KxK} \\ 1^{T} \\ -1^{T} \end{bmatrix} \begin{bmatrix} \theta \end{bmatrix} \leq \begin{bmatrix} 0 \\ 1 \\ -1 \end{bmatrix} \right\}
\end{equation*}
where $0$ is a K-vector of zeros, and $1^{T}$ is an K-vector of ones. Note that we replaced the equality constraint 
\begin{equation*}
  \sum_{i = 1}^{K} \theta_{i} = 1 
\end{equation*}
by 
\begin{align}
  \sum_{i = 1}^{K} \theta_{i} \leq 1, \sum_{i = 1}^{K} \theta_{i} \geq 1
\end{align}

We can then rewrite the first constraint of the main program such that 
\begin{equation*}
  z^{T} x \leq b - a^{T}x \Leftrightarrow \left( \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \right)^{T} x \leq b - a^{T}x
\end{equation*}

Our robustness linear problem on the uncertain variable $z$ can be written as
\begin{align}
\max_{\theta}\;\; & \left( \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \right)^T x\\
\text{s. t.}\;\; & \begin{bmatrix} -I \\ 1^T \\ -1^T \end{bmatrix} \begin{bmatrix} \theta \end{bmatrix} \leq \begin{bmatrix} 0 \\ 1 \\ -1 \end{bmatrix} \;
\end{align}

with dual problem
\begin{align}
\min_{\lambda}\;\; & \begin{bmatrix} 0 & 1 & -1 \end{bmatrix} \begin{bmatrix} \lambda \end{bmatrix}\\
\text{s. t.}\;\; & - \lambda_{i} + \lambda_{K+1} - \lambda_{K+2} = \bar{z}_{i}^T x, \forall i = 1, ..., K \\
& \lambda \geq 0 \\
\end{align}
where $\lambda \in \mathbb{R}^{K+2}$

Thus, our reduced reformulation can be written as
\begin{align}
\max_{x, \lambda}\;\; & c^T x\\
\text{s. t.}\;\; & \lambda_{K+1} - \lambda_{K+2} \leq b - a^T x \\
& - \lambda_{i} + \lambda_{K+1} - \lambda_{K+2} = \bar{z}_{i}^T x, \forall i = 1, ..., K \\
& \lambda \geq 0 \\
& 0 \leq x \leq 1 \\
\end{align}

After reformulation, we obtain:

In [ ]:
model1_red = ro.Model('Reduced_RobustCounterpart_ConvexHull')

# Define decision variable(s)
x1_red = model1_red.dvar(n)
lambda1_red = model1_red.dvar(K+2)

# Objective function
model1_red.max(c @ x1_red)

# Constraints
v = np.append(np.zeros(K), [1, -1])
WT = np.hstack(((-1)*np.eye(K,K), np.ones((K,1)), (-1)*np.ones((K,1))))
model1_red.st(v @ lambda1_red <= b - a @ x1_red)
model1_red.st(WT @ lambda1_red == Zbars.T @ x1_red)
model1_red.st(lambda1_red >= 0)
model1_red.st(x1_red >= 0)
model1_red.st(x1_red <= 1)

# Solve mdoel
model1_red.solve(my_solver)

print('The objective of Reduced Robust Counterpart is {0:0.4f}'.format(model1_red.get()))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0199s
The objective of Reduced Robust Counterpart is 3.2742


# **Exercise 2.2)**

We are interested in the budgeted uncertainty set
\begin{align}
\max_{x}\;\; & c^T x\\
\text{subject to}\;\; & (a+z)^Tx \leq b \,,\,\forall z\in\mathcal{Z}\\
& 0 \leq x \leq 1\;,
\end{align}
where
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, -1 \leq z \leq 1,\, \sum_i |z_i| \leq \Gamma\right\}$$

In [ ]:
model2_raw = ro.Model('Raw_RobustCounterpart_Budgeted') 

x=model2_raw.dvar(m) 

#Create uncertain vector
z= model2_raw.rvar(m) 
#Create uncertainty set
Z2_raw = (z<=1, z>=-1,  #each parameter is between [-1, 1] 
                  rso.norm(z,1)<=Gamma);   # Budget of uncertainty approach

model2_raw.max(c@x)
#Robustify the constraint
model2_raw.st(((a+z)@x<=b).forall(Z2_raw))
model2_raw.st(x>=0)
model2_raw.st(x<=1)
model2_raw.solve(my_solver)

optobj_raw = model2_raw.get() 
xx_raw   = x.get()

print('The objective of Raw Robust Counterpart is {0:0.4f}'.format(model2_raw.get()))


Being solved by Mosek...
Solution status: optimal
Running time: 0.0357s
The objective of Raw Robust Counterpart is 0.1314


We reformulate the above problem into a reduced tractable linear reformulation as follows:

First rewrite the uncertainty set as given in below to be able to use the equivalence idea that we discussed in class
\begin{equation*}
 \mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, \exists \Delta^+\ge 0, \exists \Delta^-\ge 0,z= \Delta^+- \Delta^-,   \Delta^++ \Delta^- \leq 1,\, \sum_i  (\Delta_i^++ \Delta_i^-) \leq \Gamma\right\}
 \end{equation*}
 \begin{equation*}
 \mathcal{Z}:= \left\{ \begin{bmatrix} \Delta^+ \in \mathbb{R}^{m} \\\Delta^-\in \mathbb{R}^{m} \end{bmatrix}  \middle|\, \begin{bmatrix} -I & 0 \\ 0 & -I \\ I & I \\ 1 & 1 \end{bmatrix}\begin{bmatrix} \Delta^+ \\\Delta^- \end{bmatrix} \leq \begin{bmatrix} 0 \\0 \\1 \\ \Gamma \end{bmatrix} \right\}
\end{equation*}


We can then rewrite the first constraint of the main program such that 
\begin{equation*}
  z^{T} x \leq b - a^{T}x \Leftrightarrow \left( \Delta^+- \Delta^- \right)^{T} x \leq b - a^{T}x
\end{equation*}

Our robustness linear problem on the uncertain variable $z$ can be written as
\begin{align}
\max_{\Delta^+,\Delta^-}\;\; & \begin{bmatrix} \Delta^+ \\\Delta^- \end{bmatrix}^T \begin{bmatrix} x \\ -x \end{bmatrix}\\
\text{s. t.}\;\; & \begin{bmatrix} -I & 0 \\ 0 & -I \\ I & I \\ 1 & 1 \end{bmatrix}\begin{bmatrix} \Delta^+ \\\Delta^- \end{bmatrix} \leq \begin{bmatrix} 0 \\0 \\1 \\ \Gamma \end{bmatrix} \;
\end{align}

with dual problem
\begin{align}
\min_{\alpha^+,\alpha^-,\gamma,\lambda}\;\; & \begin{bmatrix} 0 & 0 & 1 & \Gamma \end{bmatrix} \begin{bmatrix} \alpha^+ \\ \alpha^-\\ \gamma\\ \lambda\end{bmatrix}\\
\text{s. t.}\;\; & \begin{bmatrix} -I & 0 & I &1 \\ 0 & -I & I &1 \end{bmatrix}\begin{bmatrix} \alpha^+ \\ \alpha^-\\ \gamma\\ \lambda\end{bmatrix}=\begin{bmatrix} x \\ -x \end{bmatrix}\\
& \begin{bmatrix} \alpha^+ & \alpha^- & \gamma & \lambda \end{bmatrix}^T \geq 0 
\end{align}
where $\alpha^+,\alpha^-,\gamma \in \mathbb{R}^{m}$.

Thus, our reduced reformulation can be written as
\begin{align}
\max_{x, \gamma, \lambda}\;\; & c^T x\\
\text{s. t.}\;\; & \sum_{i = 1}^{m} \gamma_{i} + \lambda \Gamma \leq b - a^T x \\
& x_{i} \geq - \gamma_{i} - \lambda, \forall i = 1, ..., m \\
& x_{i} \leq \gamma_{i} + \lambda, \forall i = 1, ..., m \\
& 0 \leq x \leq 1 \\
& \gamma, \lambda \geq 0 
\end{align}


In [ ]:
model2_red = ro.Model('Reduced_RobustCounterpart_Budgeted')   

# Define decision variable(s)
x2_red = model2_red.dvar(n)
gamma2_red = model2_red.dvar(n)
lambda2_red = model2_red.dvar(1)

# Objective function
model2_red.max(c @ x2_red)

# Constraints
model2_red.st(sum(gamma2_red) + (lambda2_red * Gamma) <= (b - (a @ x2_red)))
model2_red.st(x2_red <= (gamma2_red + (lambda2_red * np.ones((n,1)))))
model2_red.st(x2_red >= ((-1) * gamma2_red - (lambda2_red * np.ones((n,1)))))
model2_red.st(x2_red >= 0)
model2_red.st(x2_red <= 1)
model2_red.st(gamma2_red >= 0)
model2_red.st(lambda2_red >= 0)

# Solve model
model2_red.solve(my_solver)

print('The objective of Reduced Robust Counterpart is {0:0.4f}'.format(model2_red.get()))


Being solved by Mosek...
Solution status: optimal
Running time: 0.0593s
The objective of Reduced Robust Counterpart is 0.1314


# **Exercise 2.3)**

We are interested in the CVaR uncertainty set
\begin{align}
\max_{x}\;\; & c^T x\\
\text{subject to}\;\; & (a+z)^Tx \leq b \,,\,\forall z\in\mathcal{Z}\\
& 0 \leq x \leq 1\;,
\end{align}
where
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^n\,\middle|\, \exists \theta\in\mathbb{R}^K,\, z = \sum_{i=1}^K \theta_i \bar{z}_i,\, \theta\geq 0, \, \sum_{i=1}^K \theta_i = 1,\, \theta\leq \frac{1}{K\alpha}\right\}$$


In [ ]:
model3_raw = ro.Model('Raw_RobustCounterpart_CVaR')  

# Define decision variable(s)
x3_raw = model3_raw.dvar(n)

# Define uncertain variable
z3_raw = model3_raw.rvar(n)
t3_raw = model3_raw.rvar(K)

# Define uncertainty set
Z3_raw = (z3_raw == (Zbars @t3_raw), t3_raw >= 0, sum(t3_raw) == 1, t3_raw <= (1/(K*CVaR_alpha))) 

# Objective function
model3_raw.max(c @ x3_raw)

# Constraints
model3_raw.st(((a + z3_raw) @ x3_raw <= b).forall(Z3_raw))
model3_raw.st(x3_raw >= 0)
model3_raw.st(x3_raw <= 1)

# Solve mdoel
model3_raw.solve(my_solver)

print('The objective of Raw Robust Counterpart is {0:0.4f}'.format(model3_raw.get()))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0329s
The objective of Raw Robust Counterpart is 3.4787


We reformulate the above problem into a reduced tractable linear reformulation as follows:

First rewrite the uncertainty set as 
\begin{equation*}
  \mathcal{Z}_{1} := \left\{ z = \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \in \mathbb{R}^n \middle|\, \theta \geq 0, \sum_{i = 1}^{K} \theta_{i} = 1, \theta\leq \frac{1}{K\alpha} \right\}
\end{equation*}

which we rewrite in the familiar form of $Wz \leq v$ giving
\begin{equation*}
  \mathcal{Z}_{1} := \left\{ \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \in \mathbb{R}^n \middle|\, \begin{bmatrix} -I \in \mathbb{R}^{KxK} \\ 1^{T} \\ -1^{T} \\I \in \mathbb{R}^{KxK} \end{bmatrix} \begin{bmatrix} \theta \end{bmatrix} \leq \begin{bmatrix} 0 \\ 1 \\ -1 \\ (\frac{1}{K\alpha}) 1 \end{bmatrix} \right\}
\end{equation*}
where $0$ is a K-vector of zeros, $1^{T}$ is an K-vector of ones, and $(\frac{1}{K\alpha}) 1$ is also a K-vector with value $\frac{1}{K\alpha}$ for each of its entries. Note that we replaced the equality constraint 
\begin{equation*}
  \sum_{i = 1}^{K} \theta_{i} = 1 
\end{equation*}
by 
\begin{align}
  \sum_{i = 1}^{K} \theta_{i} \leq 1, \sum_{i = 1}^{K} \theta_{i} \geq 1
\end{align}

We can then rewrite the first constraint of the main program such that 
\begin{equation*}
  z^{T} x \leq b - a^{T}x \Leftrightarrow \left( \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \right)^{T} x \leq b - a^{T}x
\end{equation*}

Our robustness linear problem on the uncertain variable $z$ can be written as
\begin{align}
\max_{\theta}\;\; & \left( \sum_{i = 1}^{K} \theta_{i} \bar{z}_{i} \right)^T x\\
\text{s. t.}\;\; & \begin{bmatrix} -I \\ 1^T \\ -1^T \\ I\end{bmatrix} \begin{bmatrix} \theta \end{bmatrix} \leq \begin{bmatrix} 0 \\ 1 \\ -1 \\ (\frac{1}{K\alpha}) 1 \end{bmatrix} \;
\end{align}

with dual problem
\begin{align}
\min_{\lambda}\;\; & \begin{bmatrix} 0 & 1 & -1 & (\frac{1}{K\alpha}) 1 \end{bmatrix} \begin{bmatrix} \lambda \end{bmatrix}\\
\text{s. t.}\;\; & - \lambda_{i} + \lambda_{K+1} - \lambda_{K+2} + \lambda_{K+2+i}= \bar{z}_{i}^T x, \forall i = 1, ..., K \\
& \lambda \geq 0 \\
\end{align}
where $\lambda \in \mathbb{R}^{2K+2}$

Thus, our reduced reformulation can be written as
\begin{align}
\max_{x, \lambda}\;\; & c^T x\\
\text{s. t.}\;\; & \lambda_{K+1} - \lambda_{K+2} +\frac{1}{K\alpha} \sum_{i = 1}^{K}\lambda_{K+2+i} \leq b - a^T x \\
& - \lambda_{i} + \lambda_{K+1} - \lambda_{K+2} + \lambda_{K+2+i}= \bar{z}_{i}^T x, \forall i = 1, ..., K \\
& \lambda \geq 0 \\
& 0 \leq x \leq 1 \\
\end{align}

After reformulation, we obtain:


In [ ]:
model3_red = ro.Model('Reduced_RobustCounterpart_CVaR')

# Define decision variable(s)
x3_red = model3_red.dvar(n)
lambda3_red = model3_red.dvar(2*K+2)

# Objective function
model3_red.max(c @ x3_red)

# Constraints
v = np.append(np.zeros(K), [1, -1])
for k in range(K):
  v=np.append(v,1/(K*CVaR_alpha))

WT = np.hstack(((-1)*np.eye(K,K), np.ones((K,1)), (-1)*np.ones((K,1)),np.eye(K,K)))
model3_red.st(v @ lambda3_red <= b - a @ x3_red)
model3_red.st(WT @ lambda3_red == Zbars.T @ x3_red)
model3_red.st(lambda3_red >= 0)
model3_red.st(x3_red >= 0)
model3_red.st(x3_red <= 1)

# Solve mdoel
model3_red.solve(my_solver)

print('The objective of Reduced Robust Counterpart is {0:0.4f}'.format(model3_red.get()))

Being solved by Mosek...
Solution status: optimal
Running time: 0.0314s
The objective of Reduced Robust Counterpart is 3.4787
